In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common import NoSuchElementException


In [36]:
# create a Chrome web driver instance
driver = webdriver.Chrome(service=Service())

# connect to the target page
driver.get(
    "https://www.booking.com/searchresults.en-gb.html?ss=Pila%2C+Valle+d%27Aosta%2C+Italy&efdco=1&label=gen173nr-10CAEoggI46AdIM1gEaLsBiAEBmAEzuAEHyAEM2AED6AEB-AEBiAIBqAIBuAKp5cXHBsACAdICJGY1MjYxNjA3LWI3MzQtNGI4My1hYzBlLWNkZTY0Njk5YzBkZNgCAeACAQ&sid=6c6b02e3cbfefa08662a237131922855&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=index&dest_id=900040971&dest_type=city&ac_position=1&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=440f9e957c14001c&ac_meta=GhA0NDBmOWU5NTdjMTQwMDFjIAEoATICZW46CnBpbGEgYW9zdGFAAEoAUAA%3D&checkin=2026-02-13&checkout=2026-02-17&group_adults=6&no_rooms=1&group_children=0"
)

# scraping logic...

In [37]:
close_items = [
    "button#onetrust-reject-all-handler",
    "button[aria-label='Dismiss sign-in info.']",
]

for i in close_items:
    try:
        # wait up to 20 seconds for the sign-in alert to appear
        close_button = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, i))
        )
        # click the close button
        close_button.click()
    except TimeoutException:
        print("Sign-in modal did not appear, continuing...")

Sign-in modal did not appear, continuing...


In [38]:
def handle_no_such_element_exception(data_extraction_task):
    try:
        return data_extraction_task()
    except NoSuchElementException as e:
        return None

In [39]:
import time
from selenium.webdriver.common.action_chains import ActionChains

# Scroll to the bottom of the page gradually
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # wait for new results to load
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # reached the bottom
    last_height = new_height

# Now get all property cards
property_items = driver.find_elements(By.CSS_SELECTOR, "[data-testid='property-card']")
items = []

In [40]:
for property_item in property_items:
    # scraping logic...
    url = handle_no_such_element_exception(
        lambda: property_item.find_element(
            By.CSS_SELECTOR, "a[data-testid='property-card-desktop-single-image']"
        ).get_attribute("href")
    )
    image = handle_no_such_element_exception(
        lambda: property_item.find_element(
            By.CSS_SELECTOR, "img[data-testid='image']"
        ).get_attribute("src")
    )
    title = handle_no_such_element_exception(
        lambda: property_item.find_element(
            By.CSS_SELECTOR, "[data-testid='title']"
        ).text
    )
    address = handle_no_such_element_exception(
        lambda: property_item.find_element(
            By.CSS_SELECTOR, "[data-testid='address']"
        ).text
    )
    distance = handle_no_such_element_exception(
        lambda: property_item.find_element(
            By.CSS_SELECTOR, "[data-testid='distance']"
        ).text
    )
    review_score = None
    review_count = None
    review_text = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid='review-score']").text)
    if review_text is not None:
        # split the review string by n
        parts = review_text.split("n")

    # process each part
    for part in parts:
        part = part.strip()
        # check if this part is a number (potential review score)
        if part.replace(".", "", 1).isdigit():
            review_score = float(part)
        # check if it contains the "reviews" string
        elif "reviews" in part:
            tokens = part.split()
            for token in tokens:
                if token.replace(",", "").isdigit():
                    review_count = int(token.replace(",", ""))
                    break

    
    description = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid='recommended-units']").text)

    price_element = handle_no_such_element_exception(lambda: (property_item.find_element(By.CSS_SELECTOR, "[data-testid='availability-rate-information']")))
    
    original_price = None
    price = None

    if price_element is not None:
        original_price = handle_no_such_element_exception(lambda: (
            price_element.find_element(By.CSS_SELECTOR, "[aria-hidden='true']:not([data-testid])").text.replace(",", "")
        ))
        price = handle_no_such_element_exception(lambda: (
            price_element.find_element(By.CSS_SELECTOR, "[data-testid='price-and-discounted-price']").text.replace(",", "")
        ))
    item = {
    "url": url,
    "image": image,
    "title": title,
    "address": address,
    "distance": distance,
    "review_score": review_score,
    "review_count": review_count,
    "description": description,
    "original_price": original_price,
    "price": price
    }
    items.append(item)

In [43]:
items

[{'url': 'https://www.booking.com/hotel/it/chambres-d-39-hotes-la-moraine-enchantee.en-gb.html?label=gen173nr-10CAEoggI46AdIM1gEaLsBiAEBmAEzuAEHyAEM2AED6AEB-AEBiAIBqAIBuAKp5cXHBsACAdICJGY1MjYxNjA3LWI3MzQtNGI4My1hYzBlLWNkZTY0Njk5YzBkZNgCAeACAQ&sid=6c6b02e3cbfefa08662a237131922855&aid=304142&ucfs=1&arphpl=1&checkin=2026-02-13&checkout=2026-02-17&dest_id=900040971&dest_type=city&group_adults=6&req_adults=6&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=b72a9f6f802200e4&srepoch=1760654464&all_sr_blocks=284141001_204029784_2_1_0%2C284141001_204029784_2_1_0%2C284141004_204029784_2_1_0&highlighted_blocks=284141001_204029784_2_1_0%2C284141001_204029784_2_1_0%2C284141004_204029784_2_1_0&matching_block_id=284141001_204029784_2_1_0&sr_pri_blocks=284141001_204029784_2_1_0__59600%2C284141001_204029784_2_1_0__59600%2C284141004_204029784_2_1_0__69600&from=searchresults',
  'image': 'https://cf.bstatic.com/xdata/images/hotel/square600/133064437.webp?k=306cf0331f20

In [42]:
driver.quit()

In [41]:
len(items)

66

In [45]:
import pandas as pd
df = pd.DataFrame(items)
df.head()

,url,image,title,address,distance,review_score,review_count,description,original_price,price
0,https://www.booking.com/hotel/it/chambres-d-39...,https://cf.bstatic.com/xdata/images/hotel/squa...,Chambres d'hôtes La Moraine Enchantée,Aosta,3.3 km from Pila,None,676.0,Recommended for your group\n1×\nSuperior Room ...,None,€ 1888
1,https://www.booking.com/hotel/it/le-volpi-luxu...,https://cf.bstatic.com/xdata/images/hotel/squa...,Le Volpi Luxury Apartments,Gressan,1.8 km from Pila,None,22.0,Recommended for your group\nSuperior Apartment...,None,€ 2219
2,https://www.booking.com/hotel/it/da-lule.en-gb...,https://cf.bstatic.com/xdata/images/hotel/squa...,da Lule,Aosta,4 km from Pila,None,15.0,Recommended for your group\nSuperior One-Bedro...,None,€ 933
3,https://www.booking.com/hotel/it/affittacamere...,https://cf.bstatic.com/xdata/images/hotel/squa...,Affittacamere Saint-Salod,Charvensod,3.1 km from Pila,None,14.0,Recommended for your group\n1×\nSuperior Doubl...,None,€ 1405
4,https://www.booking.com/hotel/it/sottosopra.en...,https://cf.bstatic.com/xdata/images/hotel/squa...,Sottosopra Alps Apartments & Rooms Sky&Bike room,Aosta,5 km from Pila,None,266.0,Recommended for your group\n1×\nDouble or Twin...,None,€ 1746


In [46]:
df_sorted = df.sort_values(by="price", ascending=True)
df_sorted.head()

,url,image,title,address,distance,review_score,review_count,description,original_price,price
18,https://www.booking.com/hotel/it/la-maison-des...,https://cf.bstatic.com/xdata/images/hotel/squa...,La Maison des Hirondelles Alloggio ad uso turi...,Saint-Pierre,7.4 km from Pila,None,9.0,Recommended for your group\nTwo-Bedroom Apartm...,€ 1099,€ 1013
7,https://www.booking.com/hotel/it/express-aosta...,https://cf.bstatic.com/xdata/images/hotel/squa...,Hotel Express Aosta East,Aosta,7.5 km from Pila,None,1854.0,Recommended for your group\n2×\nSuperior Tripl...,None,€ 1014
11,https://www.booking.com/hotel/it/clementine-ap...,https://cf.bstatic.com/xdata/images/hotel/squa...,L'Atelier du Temps - Clementine Appartements,Cogne,9.6 km from Pila,None,18.0,Recommended for your group\nTwo-Bedroom Apartm...,€ 1110,€ 1032
24,https://www.booking.com/hotel/it/la-dimora-del...,https://cf.bstatic.com/xdata/images/hotel/squa...,La Dimora delle Nevi,Aosta,6.9 km from Pila,None,58.0,Recommended for your group\nTwo-Bedroom Apartm...,None,€ 1115
9,https://www.booking.com/hotel/it/raggio-di-sol...,https://cf.bstatic.com/xdata/images/hotel/squa...,L'Atelier du Temps - Sole e Luna,Cogne,9.8 km from Pila,None,22.0,Recommended for your group\nApartment with Mou...,€ 1212,€ 1126


In [50]:
print(df_sorted['title'].head(1))

18    La Maison des Hirondelles Alloggio ad uso turi...
Name: title, dtype: object


In [55]:
df_sorted.to_excel("booking_scrape.xlsx", index=False)